In [2]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from src.utils.data_loader import load_data
import numpy as np
from sklearn.model_selection import train_test_split

# Define la ruta de datos y del modelo existente
data_dir = os.path.join(os.path.dirname(os.path.abspath('')), 'data', 'raw')
model_path = os.path.join(os.path.dirname(os.path.abspath('')), 'data', 'models', 'modelo_formas.h5')

# Carga el modelo existente
model = tf.keras.models.load_model(model_path)
print(f"Modelo cargado desde: {model_path}")

# Carga TODAS las imágenes disponibles (incluyendo las nuevas en 'triangle')
images, labels, forma_to_label = load_data(data_dir, max_images_per_class=None)
print(f"Cargadas {len(images)} imágenes de {len(forma_to_label)} clases")
print("Clases:", forma_to_label)

# Normaliza las imágenes
images = images / 255.0
num_classes = len(forma_to_label)
labels = tf.keras.utils.to_categorical(labels, num_classes=num_classes)

# Divide los datos (80% entrenamiento, 20% validación)
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)
print(f"Entrenamiento: {len(X_train)} imágenes, Validación: {len(X_val)} imágenes")

# Augmentación de datos con énfasis en triángulos (más variedad)
datagen = ImageDataGenerator(
    rotation_range=40,  # Mayor rotación para mejorar reconocimiento de triángulos en diferentes ángulos
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.3,  # Aumenta shear para formas angulares como triángulos
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,  # Flip vertical para más variedad
    fill_mode='nearest'
)
train_generator = datagen.flow(X_train, y_train, batch_size=32)
validation_generator = datagen.flow(X_val, y_val, batch_size=32)

# Recompila el modelo con una tasa de aprendizaje baja para reentrenamiento
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)  # Tasa baja para ajustar sin perder conocimiento previo
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Reentrena el modelo con los datos nuevos (más epochs para converger con data adicional)
print("Reentrenando el modelo con los datos nuevos...")
history = model.fit(train_generator, epochs=30, validation_data=validation_generator, verbose=1)

# Guarda el modelo reentrenado
model.save(model_path)
print(f"Modelo reentrenado guardado en: {model_path}")
print('Clases utilizadas:', forma_to_label)

Modelo cargado desde: C:\Users\GYGABYTE\Documents\SEMESTRE VII\InteraccionhumanoComputador\RecognizerOfGeometricShapes\data\models\modelo_formas.h5
Cargadas 22007 imágenes de 16 clases
Clases: {'circle': 0, 'cone': 1, 'cube': 2, 'cuboid': 3, 'cylinder': 4, 'ellipse': 5, 'hexagon': 6, 'octagon': 7, 'pentagon': 8, 'prism': 9, 'pyramid': 10, 'rectangle': 11, 'rhombus': 12, 'sphere': 13, 'square': 14, 'triangle': 15}
Entrenamiento: 17605 imágenes, Validación: 4402 imágenes
Reentrenando el modelo con los datos nuevos...
Epoch 1/30
551/551 ━━━━━━━━━━━━━━━━━━━━ 216s 386ms/step - accuracy: 0.6259 - loss: 3.1157 - val_accuracy: 0.6890 - val_loss: 1.0932
Epoch 2/30
551/551 ━━━━━━━━━━━━━━━━━━━━ 210s 381ms/step - accuracy: 0.6783 - loss: 1.0682 - val_accuracy: 0.7422 - val_loss: 0.7914
Epoch 3/30
551/551 ━━━━━━━━━━━━━━━━━━━━ 210s 382ms/step - accuracy: 0.7267 - loss: 0.8328 - val_accuracy: 0.7717 - val_loss: 0.6669
Epoch 4/30
551/551 ━━━━━━━━━━━━━━━━━━━━ 211s 382ms/step - accuracy: 0.7465 - loss: 

Modelo reentrenado guardado en: C:\Users\GYGABYTE\Documents\SEMESTRE VII\InteraccionhumanoComputador\RecognizerOfGeometricShapes\data\models\modelo_formas.h5
Clases utilizadas: {'circle': 0, 'cone': 1, 'cube': 2, 'cuboid': 3, 'cylinder': 4, 'ellipse': 5, 'hexagon': 6, 'octagon': 7, 'pentagon': 8, 'prism': 9, 'pyramid': 10, 'rectangle': 11, 'rhombus': 12, 'sphere': 13, 'square': 14, 'triangle': 15}


Imagen guardada en: D:\Descargas 2.0\SEMESTREVII\InteraccionHumanoComputador\PrimeraTarea\AnalisiOfSolidAndPlane\data\raw\cube\cube_164.jpg
Cámara activada. Dibuja un rectángulo alrededor del objeto con el mouse y presiona 'c' para capturar. Presiona 'q' para salir.
Forma predicha: square (Confianza: 50.03%)


¿Es correcto? (s/n, o 'q' para salir y reentrenar):  n


Selecciona la forma correcta (0-23): {'circle': 0, 'cone': 1, 'cube': 2, 'cuboid': 3, 'cylinder': 4, 'dodecahedron': 5, 'ellipse': 6, 'heptagon': 7, 'hexagon': 8, 'nonagon': 9, 'octagon': 10, 'parallelogram': 11, 'pentagon': 12, 'prism': 13, 'pyramid': 14, 'rectangle': 15, 'rhombus': 16, 'sphere': 17, 'square': 18, 'tetrahedron': 19, 'trapezoid': 20, 'triangle': 21}


...

Cámara activada. Dibuja un rectángulo alrededor del objeto con el mouse y presiona 'c' para capturar.
Forma predicha: rhombus


¿Es correcto? (s/n, o 'q' para salir):  n


Selecciona la forma correcta (0-23): {'circle': 0, 'cone': 1, 'cube': 2, 'cuboid': 3, 'cylinder': 4, 'decagon': 5, 'dodecahedron': 6, 'ellipse': 7, 'heptagon': 8, 'hexagon': 9, 'nonagon': 10, 'octagon': 11, 'octahedron': 12, 'parallelogram': 13, 'pentagon': 14, 'prism': 15, 'pyramid': 16, 'rectangle': 17, 'rhombus': 18, 'sphere': 19, 'square': 20, 'tetrahedron': 21, 'trapezoid': 22, 'triangle': 23}


Ingresa el número de la forma correcta:  0


Cámara activada. Dibuja un rectángulo alrededor del objeto con el mouse y presiona 'c' para capturar.


In [2]:
import sys
import os
sys.path.append(os.path.abspath('..'))

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from src.utils.data_loader import load_data
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Define la ruta de datos y del modelo
data_dir = os.path.join(os.path.dirname(os.path.abspath('')), 'data', 'raw')
model_path = os.path.join(os.path.dirname(os.path.abspath('')), 'data', 'models', 'modelo_formas.h5')

# Define las clases utilizadas
selected_classes = ['circle', 'cone', 'cube', 'cuboid', 'cylinder', 'ellipse', 'hexagon', 'octagon', 'pentagon', 'prism', 'pyramid', 'rectangle', 'rhombus', 'sphere', 'square', 'triangle']
num_classes = len(selected_classes)

# Carga el modelo
model = tf.keras.models.load_model(model_path)
print(f"Modelo cargado desde: {model_path}")

# Carga todos los datos disponibles (500 imágenes por clase)
images, labels, forma_to_label = load_data(data_dir, max_images_per_class=500)  # Usa todas las 500 por clase
print(f"Cargadas {len(images)} imágenes de {len(forma_to_label)} clases iniciales")
print("Clases iniciales:", forma_to_label)

# Filtra imágenes y etiquetas para las clases seleccionadas, asegurando todas las imágenes
images_selected = []
labels_selected = []
for i, forma in enumerate(forma_to_label):
    if forma in selected_classes:
        class_indices = [j for j, label in enumerate(labels) if label == i]
        images_selected.extend([images[j] for j in class_indices])
        labels_selected.extend([i] * len(class_indices))

# Crea un mapeo de índices originales a nuevos índices basados en selected_classes
selected_indices = [i for i, forma in enumerate(forma_to_label) if forma in selected_classes]
label_mapping = {i: idx for idx, i in enumerate(selected_indices)}
labels_mapped = [label_mapping.get(label, -1) for label in labels_selected]
labels_mapped = [label for label in labels_mapped if label != -1]
images_selected = [img for i, img in enumerate(images_selected) if i < len(labels_mapped)]
labels_mapped = labels_mapped[:len(images_selected)]  # Asegura misma longitud

print(f"Cargadas {len(images_selected)} imágenes de {len(selected_classes)} clases tras filtrado")
print("Clases seleccionadas:", selected_classes)

# Normaliza las imágenes
images_selected = np.array(images_selected) / 255.0
labels_one_hot = tf.keras.utils.to_categorical(labels_mapped, num_classes)

# Divide los datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(images_selected, labels_one_hot, test_size=0.2, random_state=42)
print(f"Entrenamiento: {len(X_train)} imágenes, Prueba: {len(X_test)} imágenes")

# Evalúa el modelo en el conjunto de prueba
predictions = model.predict(np.array(X_test))
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_test, axis=1)

# Métricas con manejo de todas las clases
accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='weighted', zero_division=0, labels=range(num_classes))
recall = recall_score(true_classes, predicted_classes, average='weighted', zero_division=0, labels=range(num_classes))
conf_matrix = confusion_matrix(true_classes, predicted_classes, labels=range(num_classes))

print(f"\nEvaluación del Modelo Actual:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print("Matriz de Confusión:")
print(conf_matrix)

# Comparación con el modelo anterior
previous_accuracy = 0.53  # Valor del modelo anterior con 1000 imágenes de 20 clases
print(f"\nComparación con el Modelo Anterior:")
print(f"Accuracy Anterior: {previous_accuracy:.2f}")
print(f"Accuracy Actual: {accuracy:.2f}")
if accuracy > previous_accuracy:
    print("El modelo actual mejoró en comparación con el anterior.")
elif accuracy < previous_accuracy:
    print("El modelo actual empeoró en comparación con el anterior.")
else:
    print("El modelo actual tiene el mismo rendimiento que el anterior.")

# Mejora si la precisión es baja
if accuracy < 0.8 and len(images_selected) > num_classes:
    print("\nPrecisión baja. Iniciando reentrenamiento...")
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    train_generator = datagen.flow(np.array(X_train), y_train, batch_size=32)
    validation_generator = datagen.flow(np.array(X_val), y_val, batch_size=32)

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_generator, epochs=20, validation_data=validation_generator, verbose=1)

    # Guarda el modelo mejorado
    model.save(model_path)
    print(f"Modelo mejorado guardado en: {model_path}")
else:
    print("\nPrecisión aceptable o datos insuficientes. No se requiere reentrenamiento.")

Modelo cargado desde: D:\Descargas 2.0\SEMESTREVII\InteraccionHumanoComputador\PrimeraTarea\AnalisiOfSolidAndPlane\data\models\modelo_formas.h5
Cargadas 4695 imágenes de 16 clases iniciales
Clases iniciales: {'circle': 0, 'cone': 1, 'cube': 2, 'cuboid': 3, 'cylinder': 4, 'ellipse': 5, 'hexagon': 6, 'octagon': 7, 'pentagon': 8, 'prism': 9, 'pyramid': 10, 'rectangle': 11, 'rhombus': 12, 'sphere': 13, 'square': 14, 'triangle': 15}
Cargadas 4695 imágenes de 16 clases tras filtrado
Clases seleccionadas: ['circle', 'cone', 'cube', 'cuboid', 'cylinder', 'ellipse', 'hexagon', 'octagon', 'pentagon', 'prism', 'pyramid', 'rectangle', 'rhombus', 'sphere', 'square', 'triangle']
Entrenamiento: 3756 imágenes, Prueba: 939 imágenes
30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step

Evaluación del Modelo Actual:
Accuracy: 0.87
Precision: 0.88
Recall: 0.87
Matriz de Confusión:
[[115   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0  24   0   0   0   0   0   0   0   0   7   0   2   7   0   0]
 [  0